# K-Fold CV with Universal Features + Species

## Key Insight from Leaderboard User

**"Test dataset uses locations that aren't in the training dataset"**

## Problem with Previous Approach

Previous K-Fold ensemble scored **0.50** (worse than baseline 0.51) because:
- Auxiliary pretraining taught model to predict **location-specific** features
- State classification (NSW/Tas/Vic/WA) → useless for unseen locations!
- Weather features (rainfall, temp, ET0) → location-specific climate patterns

## New Strategy: Universal Features + Species

### Phase 1 Auxiliary Tasks (IMPROVED):
✅ **KEEP:** NDVI (vegetation density - universal across all locations)  
✅ **KEEP:** Height (plant height - universal physical measurement)  
✅ **KEEP:** Season/Daylength (calendar-based - universal)  
✅ **KEEP:** Species classification (plant type - universal! Same species = similar biomass)  

❌ **REMOVE:** State classification (location-specific!)  
❌ **REMOVE:** Weather features (rainfall, temp, ET0 - location-specific!)  

## Expected Improvement

- Previous K-Fold: 0.50 (learned to recognize training locations)
- Universal features + Species: **0.52-0.54** (should generalize to new locations!)

---

In [2]:
# Cell 1: Setup

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from datetime import datetime

sns.set_style('whitegrid')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✓ Setup complete")

Device: cpu
✓ Setup complete


In [3]:
# Cell 2: Configuration

# ================================================================
# 🐛 DEBUG MODE - SET TO FALSE FOR FULL TRAINING
# ================================================================

DEBUG_MODE = False  # ← Change to False for overnight run

BATCH_SIZE = 16
TARGET_COLS = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
COMPETITION_WEIGHTS = np.array([0.1, 0.1, 0.1, 0.2, 0.5])
NUM_FOLDS = 5

if DEBUG_MODE:
    print("\n" + "🐛"*40)
    print("DEBUG MODE: 2 epochs/phase per fold")
    print("Expected time: ~20 minutes total")
    print("🐛"*40 + "\n")
else:
    print("\n" + "🚀"*40)
    print("FULL TRAINING MODE")
    print("Expected time: ~4 hours total")
    print("🚀"*40 + "\n")

def get_epochs(full):
    return 2 if DEBUG_MODE else full

# Configuration
BASE_CONFIG = {
    'phase1_epochs': get_epochs(15),
    'phase2_epochs': get_epochs(30),
    'phase1_lr': 3e-4,
    'phase2_lr': 3e-4,
    'phase2_backbone_lr': 1e-5,
    'weight_decay': 1e-4,
    'hidden_dim': 256,
    'dropout': 0.2,
    'backbone': 'resnet18'
}

print("="*80)
print("K-FOLD CV WITH UNIVERSAL FEATURES + SPECIES")
print("="*80)
print(f"\nFolds: {NUM_FOLDS}")
print(f"Model: ResNet18")
print(f"Phase 1: {BASE_CONFIG['phase1_epochs']} epochs (universal features + species)")
print(f"Phase 2: {BASE_CONFIG['phase2_epochs']} epochs")
print(f"\nAuxiliary tasks: NDVI, Height, Season, Species (NO State/Weather!)")
print(f"\n⏱️  {'DEBUG: ~20 min' if DEBUG_MODE else 'FULL: ~4 hours'}")
print("="*80)


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
FULL TRAINING MODE
Expected time: ~4 hours total
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

K-FOLD CV WITH UNIVERSAL FEATURES + SPECIES

Folds: 5
Model: ResNet18
Phase 1: 15 epochs (universal features + species)
Phase 2: 30 epochs

Auxiliary tasks: NDVI, Height, Season, Species (NO State/Weather!)

⏱️  FULL: ~4 hours


In [4]:
# Cell 3: Load Data

train_enriched = pd.read_csv('competition/train_enriched.csv')
train_enriched['Sampling_Date'] = pd.to_datetime(train_enriched['Sampling_Date'])
train_enriched['full_image_path'] = train_enriched['image_path'].apply(lambda x: f'competition/{x}')

# Encode Species
from sklearn.preprocessing import LabelEncoder
species_encoder = LabelEncoder()
train_enriched['Species_encoded'] = species_encoder.fit_transform(train_enriched['Species'])
NUM_SPECIES = len(species_encoder.classes_)

print(f"Full dataset: {len(train_enriched)} images")
print(f"Species: {NUM_SPECIES} unique classes")
print(f"Will create {NUM_FOLDS} folds of ~{len(train_enriched)//NUM_FOLDS} val images each")

# Calculate normalization on FULL dataset
target_means = torch.tensor([train_enriched[col].mean() for col in TARGET_COLS], dtype=torch.float32)
target_stds = torch.tensor([train_enriched[col].std() for col in TARGET_COLS], dtype=torch.float32)

print("\nNormalization (full dataset):")
for i, col in enumerate(TARGET_COLS):
    print(f"  {col:15s}: mean={target_means[i]:.2f}g, std={target_stds[i]:.2f}g")

# Universal features + Species
universal_features = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'daylength', 'season']

print(f"\n✓ Using UNIVERSAL features: {universal_features}")
print(f"✓ Using Species (15 classes - universal across locations)")
print("✓ Removed: State, Weather (location-specific)")

Full dataset: 357 images
Species: 15 unique classes
Will create 5 folds of ~71 val images each

Normalization (full dataset):
  Dry_Green_g    : mean=26.62g, std=25.40g
  Dry_Dead_g     : mean=12.04g, std=12.40g
  Dry_Clover_g   : mean=6.65g, std=12.12g
  GDM_g          : mean=33.27g, std=24.94g
  Dry_Total_g    : mean=45.32g, std=27.98g

✓ Using UNIVERSAL features: ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'daylength', 'season']
✓ Using Species (15 classes - universal across locations)
✓ Removed: State, Weather (location-specific)


In [5]:
# Cell 4: Forum User's Exact R² Metric

def weighted_r2_score(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Exact implementation from 22nd place leaderboard user.
    
    Key difference from sklearn:
    - Uses per-batch mean for ss_tot calculation
    - Explicitly divides by sum of weights
    
    y_true, y_pred: shape (N, 5)
    """
    weights = COMPETITION_WEIGHTS
    r2_scores = []

    for i in range(5):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]

        ss_res = np.sum((y_t - y_p) ** 2)
        ss_tot = np.sum((y_t - np.mean(y_t)) ** 2)
        r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0

        r2_scores.append(r2)

    r2_scores = np.array(r2_scores)
    weighted_r2 = np.sum(r2_scores * weights) / np.sum(weights)

    return weighted_r2, r2_scores

print("✓ Forum user's exact R² metric defined")
print("  Uses per-batch mean for ss_tot calculation")
print("  Should match Kaggle competition scoring exactly")

✓ Forum user's exact R² metric defined
  Uses per-batch mean for ss_tot calculation
  Should match Kaggle competition scoring exactly


In [6]:
# Cell 5: Dataset Classes

class NormalizedDataset(Dataset):
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds

        transform_list = [transforms.Resize((224, 224))]
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.transform = transforms.Compose(transform_list)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        return {'image': img, 'targets': targets_normalized, 'targets_original': targets}

class UniversalDataset(Dataset):
    """Dataset with UNIVERSAL features + Species."""
    def __init__(self, dataframe, target_means, target_stds, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.target_means = target_means
        self.target_stds = target_stds

        transform_list = [transforms.Resize((224, 224))]
        if augment:
            transform_list.extend([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomRotation(10),
            ])
        transform_list.extend([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.transform = transforms.Compose(transform_list)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Image
        img = Image.open(row['full_image_path']).convert('RGB')
        img = self.transform(img)
        
        # Universal features
        ndvi = torch.tensor(row['Pre_GSHH_NDVI'], dtype=torch.float32)
        height = torch.tensor(row['Height_Ave_cm'], dtype=torch.float32)
        daylength = torch.tensor(row['daylength'], dtype=torch.float32)
        season = torch.tensor(row['season'], dtype=torch.float32)
        species = torch.tensor(row['Species_encoded'], dtype=torch.long)
        
        # Targets
        targets = torch.tensor(row[TARGET_COLS].values.astype('float32'), dtype=torch.float32)
        targets_normalized = (targets - self.target_means) / self.target_stds
        
        return {
            'image': img,
            'ndvi': ndvi,
            'height': height,
            'daylength': daylength,
            'season': season,
            'species': species,
            'targets': targets_normalized,
            'targets_original': targets
        }

print("✓ Dataset classes defined")
print("✓ UniversalDataset includes: NDVI, Height, Daylength, Season, Species")

✓ Dataset classes defined
✓ UniversalDataset includes: NDVI, Height, Daylength, Season, Species


In [7]:
# Cell 6: Model with Universal Features + Species

class UniversalAuxiliaryModel(nn.Module):
    """ResNet18 with UNIVERSAL auxiliary tasks + Species classification."""
    def __init__(self, num_outputs=5, hidden_dim=256, dropout=0.2, num_species=15):
        super().__init__()
        # ResNet18 backbone
        model = models.resnet18(weights=None)  # No pretrained (avoid download)
        self.backbone = nn.Sequential(*list(model.children())[:-1])
        feature_dim = 512

        # UNIVERSAL auxiliary heads
        self.ndvi_head = nn.Linear(feature_dim, 1)
        self.height_head = nn.Linear(feature_dim, 1)
        self.daylength_head = nn.Linear(feature_dim, 1)
        self.season_head = nn.Linear(feature_dim, 1)
        self.species_head = nn.Linear(feature_dim, num_species)  # Classification head
        
        # NO State/Weather heads!

        # Biomass head
        self.biomass_head = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_outputs)
        )

    def forward(self, x, mode='biomass'):
        features = self.backbone(x).flatten(1)
        if mode == 'auxiliary':
            return {
                'ndvi': self.ndvi_head(features),
                'height': self.height_head(features),
                'daylength': self.daylength_head(features),
                'season': self.season_head(features),
                'species': self.species_head(features)
            }
        else:
            return self.biomass_head(features)

print("✓ UniversalAuxiliaryModel defined")
print("✓ Auxiliary heads: NDVI, Height, Daylength, Season, Species (15 classes)")
print("✓ Removed: State, Weather (location-specific!)")

✓ UniversalAuxiliaryModel defined
✓ Auxiliary heads: NDVI, Height, Daylength, Season, Species (15 classes)
✓ Removed: State, Weather (location-specific!)


In [8]:
# Cell 7: Training Functions

def auxiliary_loss_universal(predictions, targets):
    """Loss for UNIVERSAL features + Species."""
    # Regression losses for continuous features
    loss_ndvi = F.mse_loss(predictions['ndvi'].squeeze(), targets['ndvi'])
    loss_height = F.mse_loss(predictions['height'].squeeze(), targets['height'])
    loss_daylength = F.mse_loss(predictions['daylength'].squeeze(), targets['daylength'])
    loss_season = F.mse_loss(predictions['season'].squeeze(), targets['season'])
    
    # Classification loss for species
    loss_species = F.cross_entropy(predictions['species'], targets['species'])
    
    # Equal weighting for all 5 tasks
    total_loss = 0.2 * (loss_ndvi + loss_height + loss_daylength + loss_season + loss_species)
    
    return total_loss

def train_phase1(model, train_loader, val_loader, num_epochs, lr, fold_name):
    """Phase 1: Learn UNIVERSAL features + Species from images."""
    print(f"\nPHASE 1: UNIVERSAL AUXILIARY PRETRAINING - {fold_name}")
    print("Learning: NDVI, Height, Daylength, Season, Species (NO location features!)")
    
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = {
                'ndvi': batch['ndvi'].to(device),
                'height': batch['height'].to(device),
                'daylength': batch['daylength'].to(device),
                'season': batch['season'].to(device),
                'species': batch['species'].to(device)
            }
            optimizer.zero_grad()
            predictions = model(images, mode='auxiliary')
            loss = auxiliary_loss_universal(predictions, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}: Loss={train_loss:.4f}")

    print(f"✓ Phase 1 complete (learned universal features + species)\n")

def train_phase2(model, train_loader, val_loader, config, target_means, target_stds, fold_name):
    """Phase 2: Biomass fine-tuning."""
    print(f"PHASE 2: BIOMASS FINE-TUNING - {fold_name}")
    num_epochs = config['phase2_epochs']
    lr_head = config['phase2_lr']
    lr_backbone = config['phase2_backbone_lr']

    model = model.to(device)
    optimizer = torch.optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': lr_backbone},
        {'params': model.biomass_head.parameters(), 'lr': lr_head}
    ], weight_decay=config['weight_decay'])

    criterion = nn.MSELoss()
    best_r2 = -float('inf')

    for epoch in range(num_epochs):
        # Train
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False):
            images = batch['image'].to(device)
            targets = batch['targets'].to(device)
            optimizer.zero_grad()
            outputs = model(images, mode='biomass')
            loss = criterion(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        train_loss /= len(train_loader.dataset)

        # Validate
        model.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                targets_original = batch['targets_original']
                outputs = model(images, mode='biomass')
                outputs_denorm = outputs.cpu() * target_stds + target_means
                all_preds.append(outputs_denorm.numpy())
                all_targets.append(targets_original.numpy())

        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        
        # Use forum user's exact metric
        val_r2, _ = weighted_r2_score(all_targets, all_preds)

        print(f"Epoch {epoch+1}/{num_epochs}: Loss={train_loss:.4f}, Val R²={val_r2:+.4f}")

        if val_r2 > best_r2:
            best_r2 = val_r2
            torch.save(model.state_dict(), f'universal_Fold{fold_name}_best.pth')
            print(f"  💾 New best: R²={best_r2:+.4f}")

    print(f"\n✓ Phase 2 complete! Best R²={best_r2:+.4f}\n")
    return best_r2

print("✓ Training functions defined")
print("✓ Using forum user's exact R² metric for validation")

✓ Training functions defined
✓ Using forum user's exact R² metric for validation


In [9]:
# Cell 8: K-Fold Training Loop

fold_results = []
fold_models = []
fold_val_indices = []

kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

print("="*80)
print("STARTING K-FOLD TRAINING (UNIVERSAL FEATURES + SPECIES)")
print("="*80)
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode: {'DEBUG (2 epochs/phase)' if DEBUG_MODE else 'FULL TRAINING'}")
print()

STARTING K-FOLD TRAINING (UNIVERSAL FEATURES + SPECIES)
Started: 2025-11-06 22:34:45
Mode: FULL TRAINING



In [10]:
# Cell 9: Train All Folds

for fold_idx, (train_indices, val_indices) in enumerate(kf.split(train_enriched)):
    print(f"\n{'='*80}")
    print(f"FOLD {fold_idx+1}/{NUM_FOLDS}")
    print(f"{'='*80}")
    print(f"Train: {len(train_indices)} images, Val: {len(val_indices)} images")

    # Split data
    train_fold = train_enriched.iloc[train_indices].copy()
    val_fold = train_enriched.iloc[val_indices].copy()

    # Scale universal features only
    scaler = StandardScaler()
    train_fold[universal_features] = scaler.fit_transform(train_fold[universal_features])
    val_fold[universal_features] = scaler.transform(val_fold[universal_features])

    # Create datasets
    train_dataset_simple = NormalizedDataset(train_fold, target_means, target_stds, augment=True)
    train_dataset_universal = UniversalDataset(train_fold, target_means, target_stds, augment=True)
    val_dataset_simple = NormalizedDataset(val_fold, target_means, target_stds, augment=False)
    val_dataset_universal = UniversalDataset(val_fold, target_means, target_stds, augment=False)

    # Create model with NUM_SPECIES parameter
    model = UniversalAuxiliaryModel(
        num_outputs=5,
        hidden_dim=BASE_CONFIG['hidden_dim'],
        dropout=BASE_CONFIG['dropout'],
        num_species=NUM_SPECIES
    )

    # Create dataloaders
    train_loader_universal = DataLoader(train_dataset_universal, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader_universal = DataLoader(val_dataset_universal, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    train_loader_simple = DataLoader(train_dataset_simple, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader_simple = DataLoader(val_dataset_simple, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Phase 1: Universal auxiliary pretraining + Species
    train_phase1(
        model=model,
        train_loader=train_loader_universal,
        val_loader=val_loader_universal,
        num_epochs=BASE_CONFIG['phase1_epochs'],
        lr=BASE_CONFIG['phase1_lr'],
        fold_name=f'{fold_idx+1}'
    )

    # Phase 2: Biomass fine-tuning
    best_r2 = train_phase2(
        model=model,
        train_loader=train_loader_simple,
        val_loader=val_loader_simple,
        config=BASE_CONFIG,
        target_means=target_means,
        target_stds=target_stds,
        fold_name=f'{fold_idx+1}'
    )

    # Load best checkpoint
    model.load_state_dict(torch.load(f'universal_Fold{fold_idx+1}_best.pth'))
    model.eval()

    # Store
    fold_results.append({
        'fold': fold_idx + 1,
        'best_r2': best_r2
    })
    fold_models.append(model)
    fold_val_indices.append(val_indices)

    print(f"✅ Fold {fold_idx+1} complete! R²={best_r2:+.4f}")

print(f"\n{'='*80}")
print("K-FOLD TRAINING COMPLETE")
print(f"{'='*80}")
print(f"\nFold R² scores (using forum user's metric):")
for res in fold_results:
    print(f"  Fold {res['fold']}: R²={res['best_r2']:+.4f}")
mean_r2 = np.mean([r['best_r2'] for r in fold_results])
std_r2 = np.std([r['best_r2'] for r in fold_results])
print(f"\nMean: {mean_r2:+.4f}")
print(f"Std:  {std_r2:.4f}")


FOLD 1/5
Train: 285 images, Val: 72 images

PHASE 1: UNIVERSAL AUXILIARY PRETRAINING - 1
Learning: NDVI, Height, Daylength, Season, Species (NO location features!)


Epoch 1/15:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 1/15: Loss=1.1752


Epoch 2/15:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 2/15: Loss=0.9571


...

Epoch 28/30: Loss=0.4015, Val R²=+0.2465


Epoch 29/30:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 29/30: Loss=0.4238, Val R²=+0.3052


Epoch 30/30:   0%|          | 0/18 [00:00<?, ?it/s]

Epoch 30/30: Loss=0.4220, Val R²=+0.2800

✓ Phase 2 complete! Best R²=+0.3763

✅ Fold 5 complete! R²=+0.3763

K-FOLD TRAINING COMPLETE

Fold R² scores (using forum user's metric):
  Fold 1: R²=+0.4146
  Fold 2: R²=+0.4055
  Fold 3: R²=+0.6126
  Fold 4: R²=+0.4763
  Fold 5: R²=+0.3763

Mean: +0.4571
Std:  0.0843


In [11]:
# Cell 10: Ensemble Evaluation

print("\n" + "="*80)
print("ENSEMBLE VALIDATION")
print("="*80)

all_ensemble_preds = []
all_ensemble_targets = []

for fold_idx, val_indices in enumerate(fold_val_indices):
    val_fold = train_enriched.iloc[val_indices]
    val_dataset = NormalizedDataset(val_fold, target_means, target_stds, augment=False)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Get predictions from all models
    fold_preds = []
    for model in fold_models:
        model = model.to(device)
        model.eval()
        preds = []
        with torch.no_grad():
            for batch in val_loader:
                images = batch['image'].to(device)
                outputs = model(images, mode='biomass')
                outputs_denorm = outputs.cpu() * target_stds + target_means
                preds.append(outputs_denorm.numpy())
        fold_preds.append(np.vstack(preds))

    # Average predictions
    ensemble_preds = np.mean(fold_preds, axis=0)
    targets = val_fold[TARGET_COLS].values

    all_ensemble_preds.append(ensemble_preds)
    all_ensemble_targets.append(targets)

# Combine all folds
all_ensemble_preds = np.vstack(all_ensemble_preds)
all_ensemble_targets = np.vstack(all_ensemble_targets)

# Calculate ensemble R² using forum user's metric
ensemble_r2, ensemble_per_target = weighted_r2_score(all_ensemble_targets, all_ensemble_preds)

print(f"\n{'='*80}")
print("ENSEMBLE RESULTS")
print(f"{'='*80}")
print(f"\nEnsemble R² ({NUM_FOLDS} models averaged): {ensemble_r2:+.4f}")
print(f"Mean single model R²: {mean_r2:+.4f}")
print(f"Ensemble improvement: {ensemble_r2 - mean_r2:+.4f}")

print(f"\n📊 vs PREVIOUS ATTEMPTS:")
print(f"  Baseline (single ResNet18): Kaggle=+0.51")
print(f"  K-Fold with location features: Kaggle=+0.50 (WORSE!)")
print(f"  This (universal features + Species): Val={ensemble_r2:+.4f}")

print(f"\nPer-target R² (Ensemble):")
for i, col in enumerate(TARGET_COLS):
    print(f"  {col:15s}: {ensemble_per_target[i]:+.4f}")

print(f"\n✅ Ensemble ready for Kaggle submission!")
print(f"   Expected Kaggle: 0.52-0.54 (should beat baseline 0.51!)")
print(f"   Species is universal - same species has similar biomass anywhere!")


ENSEMBLE VALIDATION

ENSEMBLE RESULTS

Ensemble R² (5 models averaged): +0.6975
Mean single model R²: +0.4571
Ensemble improvement: +0.2405

📊 vs PREVIOUS ATTEMPTS:
  Baseline (single ResNet18): Kaggle=+0.51
  K-Fold with location features: Kaggle=+0.50 (WORSE!)
  This (universal features + Species): Val=+0.6975

Per-target R² (Ensemble):
  Dry_Green_g    : +0.7621
  Dry_Dead_g     : +0.4153
  Dry_Clover_g   : +0.7616
  GDM_g          : +0.7486
  Dry_Total_g    : +0.7078

✅ Ensemble ready for Kaggle submission!
   Expected Kaggle: 0.52-0.54 (should beat baseline 0.51!)
   Species is universal - same species has similar biomass anywhere!


---

## Summary

### Key Changes from Previous K-Fold:

1. **Removed Location-Specific Features**
   - ❌ State classification (NSW/Tas/Vic/WA)
   - ❌ Weather features (rainfall, temp, ET0)

2. **Kept Universal Features**
   - ✅ NDVI (vegetation density)
   - ✅ Height (plant height)
   - ✅ Daylength (calendar-based)
   - ✅ Season (calendar-based)
   - ✅ Species (15 classes - universal! Same species = similar biomass anywhere)

3. **Used Forum User's Exact R² Metric**
   - Should match Kaggle scoring exactly

### Expected Outcome:

- Previous K-Fold: 0.50 (learned location-specific patterns)
- This approach: **0.52-0.54** (generalizes to new locations)

### Why Species is Universal:

- **State** = location-specific (NSW plants grow in NSW conditions)
- **Weather** = location-specific (NSW rainfall doesn't help predict Tasmanian biomass)
- **Species** = universal (Ryegrass biomass characteristics are similar everywhere!)

### Next Steps:

1. If DEBUG test passes → Set `DEBUG_MODE = False` and run overnight
2. Create Kaggle submission with ensemble
3. Compare with baseline (0.51) and previous K-Fold (0.50)
4. If successful, confirms that removing State/Weather was the key!